In [2]:
import pandas as pd

In [3]:
dataset_food_inflation_rate = pd.read_csv('./data/food_inflation_rate.csv')

In [4]:
dataset_food_inflation_rate

,Domain Code,Domain,Area Code (M49),Area,Year Code,Year,Item Code,Item,Months Code,Months,Element Code,Element,Unit,Value,Flag,Flag Description,Note
0,CP,Consumer Price Indices,248,Åland Islands,2012,2012,23014,Food price inflation,7001,January,6121,Value,%,2.607987,E,Estimated value,NaN
1,CP,Consumer Price Indices,248,Åland Islands,2012,2012,23014,Food price inflation,7002,February,6121,Value,%,3.896104,E,Estimated value,NaN
2,CP,Consumer Price Indices,248,Åland Islands,2012,2012,23014,Food price inflation,7003,March,6121,Value,%,2.967121,E,Estimated value,NaN
3,CP,Consumer Price Indices,248,Åland Islands,2012,2012,23014,Food price inflation,7004,April,6121,Value,%,2.149682,E,Estimated value,NaN
4,CP,Consumer Price Indices,248,Åland Islands,2012,2012,23014,Food price inflation,7005,May,6121,Value,%,3.559871,E,Estimated value,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,CP,Consumer Price Indices,826,United Kingdom of Great Britain and Northern I...,2023,2023,23014,Food price inflation,7008,August,6121,Value,%,13.609467,E,Estimated value,NaN
6038,CP,Consumer Price Indices,826,United Kingdom of Great Britain and Northern I...,2023,2023,23014,Food price inflation,7009,September,6121,Value,%,12.207358,E,Estimated value,NaN
6039,CP,Consumer Price Indices,826,United Kingdom of Great Britain and Northern I...,2023,2023,23014,Food price inflation,7010,October,6121,Value,%,10.163934,E,Estimated value,NaN
6040,CP,Consumer Price Indices,826,United Kingdom of Great Britain and Northern I...,2023,2023,23014,Food price inflation,7011,November,6121,Value,%,9.238250,E,Estimated value,NaN


In [5]:
dataset_raw_gdp = pd.read_csv('./data/raw_gdp.csv')

In [6]:
dataset_raw_gdp

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,MK,Macro Indicators,8,Albania,6110,Value US$,22008,Gross Domestic Product,2012,2012,million USD,1.231978e+04,X,Figure from international organizations,NaN
1,MK,Macro Indicators,8,Albania,6110,Value US$,22008,Gross Domestic Product,2013,2013,million USD,1.277628e+04,X,Figure from international organizations,NaN
2,MK,Macro Indicators,8,Albania,6110,Value US$,22008,Gross Domestic Product,2014,2014,million USD,1.322824e+04,X,Figure from international organizations,NaN
3,MK,Macro Indicators,8,Albania,6110,Value US$,22008,Gross Domestic Product,2015,2015,million USD,1.138692e+04,X,Figure from international organizations,NaN
4,MK,Macro Indicators,8,Albania,6110,Value US$,22008,Gross Domestic Product,2016,2016,million USD,1.186136e+04,X,Figure from international organizations,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6110,Value US$,22008,Gross Domestic Product,2019,2019,million USD,2.851407e+06,X,Figure from international organizations,NaN
511,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6110,Value US$,22008,Gross Domestic Product,2020,2020,million USD,2.697807e+06,X,Figure from international organizations,NaN
512,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6110,Value US$,22008,Gross Domestic Product,2021,2021,million USD,3.141506e+06,X,Figure from international organizations,NaN
513,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6110,Value US$,22008,Gross Domestic Product,2022,2022,million USD,3.089073e+06,X,Figure from international organizations,NaN


In [7]:
dataset_gdp_growth_rate = pd.read_csv('./data/gdp_growth_rate.csv')

In [8]:
dataset_gdp_growth_rate

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,MK,Macro Indicators,8,Albania,6129,Annual growth US$,22008,Gross Domestic Product,2012,2012,%,-4.430218,X,Figure from international organizations,NaN
1,MK,Macro Indicators,8,Albania,6129,Annual growth US$,22008,Gross Domestic Product,2013,2013,%,3.705432,X,Figure from international organizations,NaN
2,MK,Macro Indicators,8,Albania,6129,Annual growth US$,22008,Gross Domestic Product,2014,2014,%,3.537487,X,Figure from international organizations,NaN
3,MK,Macro Indicators,8,Albania,6129,Annual growth US$,22008,Gross Domestic Product,2015,2015,%,-13.919579,X,Figure from international organizations,NaN
4,MK,Macro Indicators,8,Albania,6129,Annual growth US$,22008,Gross Domestic Product,2016,2016,%,4.166459,X,Figure from international organizations,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6129,Annual growth US$,22008,Gross Domestic Product,2019,2019,%,-0.694212,X,Figure from international organizations,NaN
511,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6129,Annual growth US$,22008,Gross Domestic Product,2020,2020,%,-5.386835,X,Figure from international organizations,NaN
512,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6129,Annual growth US$,22008,Gross Domestic Product,2021,2021,%,16.446678,X,Figure from international organizations,NaN
513,MK,Macro Indicators,826,United Kingdom of Great Britain and Northern I...,6129,Annual growth US$,22008,Gross Domestic Product,2022,2022,%,-1.669055,X,Figure from international organizations,NaN


In [9]:
dataset_producer_prices = pd.read_csv('./data/producer_prices.csv')

In [10]:
dataset_producer_prices

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
0,PP,Producer Prices,8,Albania,5532,Producer Price (USD/tonne),01341,Apples,2012,2012,7021,Annual value,USD,369.7,A,Official figure
1,PP,Producer Prices,8,Albania,5532,Producer Price (USD/tonne),01341,Apples,2013,2013,7021,Annual value,USD,492.1,A,Official figure
2,PP,Producer Prices,8,Albania,5532,Producer Price (USD/tonne),01341,Apples,2014,2014,7021,Annual value,USD,474.8,A,Official figure
3,PP,Producer Prices,8,Albania,5532,Producer Price (USD/tonne),01341,Apples,2015,2015,7021,Annual value,USD,389.0,A,Official figure
4,PP,Producer Prices,8,Albania,5532,Producer Price (USD/tonne),01341,Apples,2016,2016,7021,Annual value,USD,381.3,A,Official figure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18671,PP,Producer Prices,826,United Kingdom of Great Britain and Northern I...,5532,Producer Price (USD/tonne),0111,Wheat,2018,2018,7021,Annual value,USD,217.0,A,Official figure
18672,PP,Producer Prices,826,United Kingdom of Great Britain and Northern I...,5532,Producer Price (USD/tonne),0111,Wheat,2019,2019,7021,Annual value,USD,188.7,A,Official figure
18673,PP,Producer Prices,826,United Kingdom of Great Britain and Northern I...,5532,Producer Price (USD/tonne),0111,Wheat,2020,2020,7021,Annual value,USD,212.5,A,Official figure
18674,PP,Producer Prices,826,United Kingdom of Great Britain and Northern I...,5532,Producer Price (USD/tonne),0111,Wheat,2021,2021,7021,Annual value,USD,275.7,A,Official figure


In [12]:
dataset_overall_inflation = pd.read_excel('./data/overall_inflation.xlsx')

In [13]:
columns_to_drop = [f'Unnamed: {i}' for i in range(2, 25) if i % 2 == 0]
columns_to_drop

['Unnamed: 2',
 'Unnamed: 4',
 'Unnamed: 6',
 'Unnamed: 8',
 'Unnamed: 10',
 'Unnamed: 12',
 'Unnamed: 14',
 'Unnamed: 16',
 'Unnamed: 18',
 'Unnamed: 20',
 'Unnamed: 22',
 'Unnamed: 24']

In [14]:
dataset_overall_inflation = dataset_overall_inflation[12:51]

In [15]:
dataset_overall_inflation = dataset_overall_inflation.drop(columns=columns_to_drop)

In [16]:
columns_to_stay = [f'Unnamed: {i}' for i in range(1, 25) if i % 2 == 1]

In [17]:
dict_rename_columns = {}

In [18]:
for prev_name, year in zip(columns_to_stay, range(2012, 2024)):
    dict_rename_columns[prev_name] = year

In [19]:
dict_rename_columns

{'Unnamed: 1': 2012,
 'Unnamed: 3': 2013,
 'Unnamed: 5': 2014,
 'Unnamed: 7': 2015,
 'Unnamed: 9': 2016,
 'Unnamed: 11': 2017,
 'Unnamed: 13': 2018,
 'Unnamed: 15': 2019,
 'Unnamed: 17': 2020,
 'Unnamed: 19': 2021,
 'Unnamed: 21': 2022,
 'Unnamed: 23': 2023}

In [20]:
dict_rename_columns['Data extracted on 22/07/2024 23:21:37 from [ESTAT]'] = 'Area'

In [21]:
dataset_overall_inflation = dataset_overall_inflation.rename(columns=dict_rename_columns)

In [22]:
dataset_overall_inflation

,Area,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
12,Belgium,2.6,1.2,0.5,0.6,1.8,2.2,2.3,1.2,0.4,3.2,10.3,2.3
13,Bulgaria,2.4,0.4,-1.6,-1.1,-1.3,1.2,2.6,2.5,1.2,2.8,13,8.6
14,Czechia,3.5,1.4,0.4,0.3,0.6,2.4,2,2.6,3.3,3.3,14.8,12
15,Denmark,2.4,0.5,0.4,0.2,0,1.1,0.7,0.7,0.3,1.9,8.5,3.4
16,Germany,2.2,1.6,0.8,0.7,0.4,1.7,1.9,1.4,0.4,3.2,8.7,6
17,Estonia,4.2,3.2,0.5,0.1,0.8,3.7,3.4,2.3,-0.6,4.5,19.4,9.1
18,Ireland,1.9,0.5,0.3,0,-0.2,0.3,0.7,0.9,-0.5,2.4,8.1,5.2
19,Greece,1,-0.9,-1.4,-1.1,0,1.1,0.8,0.5,-1.3,0.6,9.3,4.2
20,Spain,2.4,1.5,-0.2,-0.6,-0.3,2,1.7,0.8,-0.3,3,8.3,3.4
21,France,2.2,1,0.6,0.1,0.3,1.2,2.1,1.3,0.5,2.1,5.9,5.7


In [24]:
dataset_overall_inflation = dataset_overall_inflation.melt(id_vars=['Area'], var_name='Year', value_name='Overall Inflation Rate')